# 範例 : (Kaggle)房價預測
***
- 以下用房價預測資料, 觀查標籤編碼與獨編碼熱的影響

# [重點]
- 有次序性類別資料 適合使用Label Encoder 例:年齡、男女
- 名目型類別資料 適合使用 One Hot Encoder 例:縣市、名字

# [教學目標]
- 以下用房價預測資料, 觀查標籤編碼與獨編碼熱的影響

# [範例重點]
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 線性迴歸分數 / 線性迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 梯度提升樹分數 / 梯度提升樹時間上, 分別有什麼影響 (In[5], Out[5], In[6], Out[6]) 

In [1]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

data_path = '../data/'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')

train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
        
print("%d  Object Features : %s" %(len(object_features), object_features))
#print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

43  Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [3]:
# 標籤編碼 + 線性迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()

label_linear_shape = train_X.shape
label_linear_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
label_linear_time = time.time()-start
print("shape : {s}".format(s=label_linear_shape))
print("score : {s}".format(s=label_linear_score))
print("time : {t} sec".format(t=label_linear_time))

#print(f'shape : {train_X.shape}')
#print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
#print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.66156068668513
time : 0.031229496002197266 sec


In [4]:
# 獨熱編碼 + 線性迴歸
df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()

OneHot_linear_shape = train_X.shape
OneHot_linear_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
OneHot_linear_time = time.time()-start
print("shape : {s}".format(s=OneHot_linear_shape))
print("score : {s}".format(s=OneHot_linear_score))
print("time : {t} sec".format(t=OneHot_linear_time))

#print(f'shape : {train_X.shape}')
#print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
#print(f'time : {time.time() - start} sec')

shape : (1460, 274)
score : -8.09181695237409e+22
time : 0.1278553009033203 sec


In [5]:
# 標籤編碼 + 梯度提升樹
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()

label_tree_shape = train_X.shape
label_tree_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
label_tree_time = time.time()-start
print("shape : {s}".format(s=label_tree_shape))
print("score : {s}".format(s=label_tree_score))
print("time : {t} sec".format(t=label_tree_time))

#print(f'shape : {train_X.shape}')
#print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
#print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.7781813187845031
time : 0.862450361251831 sec


In [6]:
# 獨熱編碼 + 梯度提升樹
df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()

OneHot_tree_shape = train_X.shape
OneHot_tree_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
OneHot_tree_time = time.time()-start
print("shape : {s}".format(s=OneHot_tree_shape))
print("score : {s}".format(s=OneHot_tree_score))
print("time : {t} sec".format(t=OneHot_tree_time))

#print(f'shape : {train_X.shape}')
#print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
#print(f'time : {time.time() - start} sec')

shape : (1460, 274)
score : 0.7931143867500909
time : 3.4639012813568115 sec


In [7]:
data = {'Shape (features)':[label_linear_shape[1], OneHot_linear_shape[1], label_tree_shape[1], OneHot_tree_shape[1]],
       'Score':[label_linear_score, OneHot_linear_score, label_tree_score, OneHot_tree_score],
       'Time (sec)':[label_linear_time, OneHot_linear_time, label_tree_time, OneHot_tree_time],
       'Method':['LabelEncoder & LinearRegression', 'OneHotEncoder & LinearRegression',
                 'LabelEncoder & GradientBoostingRegressor', 'OneHotEncoder & GradientBoostingRegressor']}
sheet = pd.DataFrame(data)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sheet.set_index('Method', inplace=True)
sheet

,Score,Shape (features),Time (sec)
Method,,,
LabelEncoder & LinearRegression,0.662,43,0.031
OneHotEncoder & LinearRegression,-80918169523740901113856.000,274,0.128
LabelEncoder & GradientBoostingRegressor,0.778,43,0.862
OneHotEncoder & GradientBoostingRegressor,0.793,274,3.464


# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[10], Out[10], In[11], Out[11]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?


**Your answer:** *One Hot Encoder將原本43個特徵 增價至 274個*


*在範例中 線性回歸利用One Hot Encoder反而會造成預測錯誤 且 處理時間也增加5倍之多*


*而在梯度提升樹使用One Hot Encoder則可以小幅提升預測結果  0.778 -> 0.792 但 處理時間也增加3倍左右*

In [8]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [9]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
        
print("%d Numeric Features : %s" %(len(object_features), object_features))
#print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)


**Your answer:** *One Hot Encoder將原本5個特徵 增價至 2429個 將近500倍*


*羅吉斯迴歸使用One Hot Encoder可以提升預測結果  0.780 -> 0.800 2% 但 處理時間也增加6倍左右*


*同樣比較線性回歸 以及 梯度提升樹 使用One Hot Encoder 都有持平或小幅提升*


*但是鐵達尼號屬於分類問題(活 or 死) 適合羅吉斯迴歸 以至於正確率有80%*


*房價預測屬於回歸問題 所以適合以梯度提升樹 在分類問題只有40%準確度*

In [10]:
# 標籤編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()

label_log_shape = train_X.shape
label_log_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
label_log_time = time.time()-start
print("shape : {s}".format(s=label_log_shape))
print("score : {s}".format(s=label_log_score))
print("time : {t} sec".format(t=label_log_time))

shape : (891, 5)
score : 0.780004837244799
time : 0.03319597244262695 sec


In [11]:
# 獨熱編碼 + 羅吉斯迴歸

"""
Your Code Here
"""
df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()

OneHot_log_shape = train_X.shape
OneHot_log_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
OneHot_log_time = time.time()-start
print("shape : {s}".format(s=OneHot_log_shape))
print("score : {s}".format(s=OneHot_log_score))
print("time : {t} sec".format(t=OneHot_log_time))

shape : (891, 2429)
score : 0.8013346043513216
time : 0.1864161491394043 sec


In [12]:
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()

label_linear_shape = train_X.shape
label_linear_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
label_linear_time = time.time()-start


df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()

label_tree_shape = train_X.shape
label_tree_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
label_tree_time = time.time()-start

In [13]:
df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = LinearRegression()
start = time.time()

OneHot_linear_shape = train_X.shape
OneHot_linear_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
OneHot_linear_time = time.time()-start

df_temp = pd.get_dummies(df)
train_X = df_temp[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()

OneHot_tree_shape = train_X.shape
OneHot_tree_score = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
OneHot_tree_time = time.time()-start

In [14]:
data = {'Shape (features)':[label_log_shape[1], OneHot_log_shape[1],label_linear_shape[1], OneHot_linear_shape[1], label_tree_shape[1], OneHot_tree_shape[1]],
       'Score':[label_log_score, OneHot_log_score, label_linear_score, OneHot_linear_score, label_tree_score, OneHot_tree_score],
       'Time (sec)':[label_log_time, OneHot_log_time, label_linear_time, OneHot_linear_time, label_tree_time, OneHot_tree_time],
       'Method':['LabelEncoder & LogisticRegression', 'OneHotEncoder & LogisticRegression', 'LabelEncoder & LinearRegression', 'OneHotEncoder & LinearRegression',
                 'LabelEncoder & GradientBoostingRegressor', 'OneHotEncoder & GradientBoostingRegressor']}
sheet = pd.DataFrame(data)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sheet.set_index('Method', inplace=True)
sheet

,Score,Shape (features),Time (sec)
Method,,,
LabelEncoder & LogisticRegression,0.780,5,0.033
OneHotEncoder & LogisticRegression,0.801,2429,0.186
LabelEncoder & LinearRegression,0.335,5,0.020
OneHotEncoder & LinearRegression,0.413,2429,1.322
LabelEncoder & GradientBoostingRegressor,0.378,5,0.223
OneHotEncoder & GradientBoostingRegressor,0.375,2429,8.432
